In [1]:
import pandas as pd
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

/data2/home/avneeshg/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
[nltk_data] Downloading package stopwords to
[nltk_data]     /data2/home/avneeshg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
message = pd.read_csv('SMSSpamCollection.txt', sep='\t', names=['label', 'message'])

In [7]:
message

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [10]:
ps = PorterStemmer()
corpus = []

In [11]:
for i in range(0, len(message)):
    review = re.sub('[^a-zA-Z]', ' ', message['message'][i])
    review  = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]

    review = ' '.join(review)
    corpus.append(review)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)
X = cv.fit_transform(corpus).toarray()

y = pd.get_dummies(message['label'])
y = y.iloc[:,1].values

In [14]:
y

array([False, False,  True, ..., False, False, False])

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [17]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [18]:
y_pred = spam_detect_model.predict(X_test)

In [20]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_m = confusion_matrix(y_test, y_pred)
confusion_m

array([[946,   9],
       [  8, 152]])

In [22]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9847533632286996